In [77]:
import pandas as pd
import numpy as np
import os
import time
import gc
import matplotlib.pyplot as plt
import pickle
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import roc_auc_score,f1_score
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from scipy import sparse

import seaborn as sns
from datetime import *
from functools import reduce
from sklearn.metrics import roc_curve
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn import feature_selection
import datetime

import xgboost as xgb
from sklearn.feature_selection import VarianceThreshold
from catboost import CatBoostClassifier as cb
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import precision_score, recall_score,f1_score
from lightgbm import *
plt.rcParams['font.sans-serif'] = ['SimHei']
pd.set_option('display.max_info_columns', 500)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_row', 300)
pd.set_option('display.float_format', lambda x: ' %.5f' % x)


In [78]:
train_dir = "./data/1_预处理/train/"
test_dir = "./data/1_预处理/A/"

特征_train_dir = "./data/2_特征/train/"
特征_test_dir = "./data/2_特征/A/"

In [79]:
# 创建 df_train 和 df_test
date_range = pd.date_range(
    start='2014-01-01 00:00:00',
    end='2019-12-31 23:00:00',  # 包含最后一日的23:00
    freq='H'  # 每小时频率
)
df_train = pd.DataFrame(date_range, columns=['时间'])
date_range = pd.date_range(
    start='2020-01-01 00:00:00',
    end='2021-12-31 23:00:00',  # 包含最后一日的23:00
    freq='H'  # 每小时频率
)
df_test = pd.DataFrame(date_range, columns=['时间'])

In [80]:
for root, ds, fs in os.walk(特征_train_dir):
    for f in fs:
        print(f)
        with open(特征_train_dir + f , "rb" ) as file :
            df_train = df_train.merge(pickle.load(file) , on='时间' , how = 'left')
print("--------------------------------------------")
for root, ds, fs in os.walk(特征_test_dir):
    for f in fs:
        print(f)
        with open(特征_test_dir + f , "rb" ) as file :
            df_test = df_test.merge(pickle.load(file) , on='时间' , how = 'left')

1~36小时滑窗特征.pkl
基础特征.pkl
年月日季节.pkl
广东哥特征.pkl
--------------------------------------------
1~36小时滑窗特征.pkl
基础特征.pkl
年月日季节.pkl
广东哥特征.pkl


In [81]:
# 加上标签
with open(train_dir + '入库流量.pkl' , "rb" ) as file :
    df_train = df_train.merge(pickle.load(file) , on='时间' , how = 'left')

In [82]:
# 1、发现2014-06-30 01:00:00到2014-06-30 23:00:00这一天的FLAG是缺失的
df_train = df_train[~df_train['FLAG'].isna()].reset_index(drop=True).copy()

In [83]:
feature_selected = [i for i in df_train.columns if i not in ['时间' , 'FLAG']]

In [84]:
len(feature_selected)

2606

In [85]:
# df_剔除共线性特征 = corr_filter(df_train.drop(['时间' , 'FLAG'] , axis=1) , 0.95)
# feature_selected = df_剔除共线性特征

In [86]:
feature_selected = ['下保村雨量_max_36h',
 '下保村雨量_max_48h',
 '下保村雨量_std_12h',
 '下保村雨量_std_36h',
 '下保村雨量_std_48h',
 '下墩村雨量_std_12h',
 '下洋坂雨量_std_3h',
 '下洋坂雨量_std_6h',
 '下路岭雨量_min_24h',
 '东上村雨量_sum_48h',
 '九丘雨量_max_24h',
 '厚德村雨量_min_48h',
 '双鲤村雨量_min_24h',
 '后村雨量_min_36h',
 '均中村雨量_max_12h',
 '均中村雨量_max_36h',
 '均中村雨量_max_48h',
 '均中村雨量_std_3h',
 '均中村雨量_std_36h',
 '均中村雨量_std_48h',
 '均中村雨量_sum_48h',
 '夏阳乡雨量_min_36h',
 '外洋村雨量_max_12h',
 '外洋村雨量_max_48h',
 '外洋村雨量_std_12h',
 '外洋村雨量_std_48h',
 '外洋村雨量_sum_48h',
 '大华镇雨量_max_48h',
 '大华镇雨量_min_24h',
 '大华镇雨量_sum_48h',
 '大口窑雨量_max_24h',
 '大口窑雨量_max_36h',
 '大口窑雨量_std_12h',
 '大口窑雨量_std_24h',
 '大口窑雨量_std_36h',
 '大口窑雨量_std_48h',
 '大横镇曲楼甲新农村雨量_max_36h',
 '大横镇曲楼甲新农村雨量_max_48h',
 '大横镇曲楼甲新农村雨量_std_3h',
 '大横镇曲楼甲新农村雨量_std_6h',
 '大横镇曲楼甲新农村雨量_std_24h',
 '大横镇曲楼甲新农村雨量_std_48h',
 '大横镇曲楼甲新农村雨量_sum_36h',
 '大横镇曲楼甲新农村雨量_sum_48h',
 '天心岛花园雨量_max_48h',
 '天心岛花园雨量_sum_36h',
 '安砂镇雨量_max_48h',
 '安砂镇雨量_std_24h',
 '富屯溪流域面雨量_max_6h',
 '富屯溪流域面雨量_max_36h',
 '富屯溪流域面雨量_max_48h',
 '富屯溪流域面雨量_min_6h',
 '富屯溪流域面雨量_min_12h',
 '富屯溪流域面雨量_sum_36h',
 '富屯溪流域面雨量_sum_48h',
 '小华村雨量_max_3h',
 '小华村雨量_min_24h',
 '小西门头社区雨量_max_48h',
 '山下乡雨量_max_12h',
 '山下乡雨量_max_48h',
 '山下乡雨量_std_36h',
 '山下乡雨量_sum_12h',
 '山下乡雨量_sum_48h',
 '山坊雨量_max_6h',
 '山坊雨量_max_48h',
 '山坊雨量_sum_36h',
 '山坊雨量_sum_48h',
 '岭坪雨量_max_48h',
 '岭坪雨量_min_6h',
 '岭坪雨量_std_3h',
 '岭坪雨量_std_6h',
 '岭坪雨量_std_36h',
 '岭坪雨量_std_48h',
 '岭坪雨量_sum_48h',
 '年坂山雨量_std_48h',
 '库区流域面雨量_max_48h',
 '建溪流域面雨量_max_48h',
 '建溪流域面雨量_min_36h',
 '建溪流域面雨量_min_48h',
 '建溪流域面雨量_sum_48h',
 '文峰村雨量_std_6h',
 '文峰村雨量_std_24h',
 '文峰村雨量_std_36h',
 '新桥头雨量_max_48h',
 '新桥头雨量_std_6h',
 '新桥头雨量_std_48h',
 '新桥头雨量_sum_24h',
 '新桥头雨量_sum_48h',
 '新源村雨量_max_36h',
 '新源村雨量_max_48h',
 '新源村雨量_std_6h',
 '新源村雨量_std_36h',
 '新源村雨量_std_48h',
 '新源村雨量_sum_36h',
 '新源村雨量_sum_48h',
 '景山新村雨量_min_48h',
 '杉坜雨量_std_3h',
 '杉坜雨量_std_6h',
 '杉坜雨量_std_36h',
 '杉坜雨量_std_48h',
 '杉坜雨量_sum_48h',
 '林后岗雨量_max_48h',
 '林后岗雨量_min_24h',
 '林后岗雨量_min_36h',
 '林后岗雨量_min_48h',
 '林后岗雨量_std_3h',
 '林后岗雨量_std_12h',
 '林后岗雨量_std_48h',
 '林后岗雨量_sum_24h',
 '林后岗雨量_sum_36h',
 '林后岗雨量_sum_48h',
 '柯坑村雨量_max_36h',
 '柯坑村雨量_max_48h',
 '桐木村雨量_max_12h',
 '桐木村雨量_max_24h',
 '桐木村雨量_max_36h',
 '桐木村雨量_std_36h',
 '桐木村雨量_std_48h',
 '桐木村雨量_sum_48h',
 '桥南村雨量_max_6h',
 '桥南村雨量_max_12h',
 '桥南村雨量_max_48h',
 '桥南村雨量_min_24h',
 '桥南村雨量_min_36h',
 '桥南村雨量_min_48h',
 '桥南村雨量_std_3h',
 '桥南村雨量_std_6h',
 '桥南村雨量_std_12h',
 '桥南村雨量_std_48h',
 '桥南村雨量_sum_36h',
 '桥南村雨量_sum_48h',
 '梅仙镇雨量_min_36h',
 '梅仙镇雨量_std_48h',
 '梅仙镇雨量_sum_48h',
 '梅园小区雨量_max_24h',
 '梅园小区雨量_min_24h',
 '梅园小区雨量_std_24h',
 '梅园小区雨量_sum_24h',
 '梅山镇雨量_min_48h',
 '梅树湾新村雨量_max_48h',
 '梅树湾新村雨量_sum_48h',
 '樟树村雨量_max_36h',
 '樟树村雨量_max_48h',
 '樟树村雨量_std_3h',
 '樟树村雨量_std_12h',
 '樟树村雨量_sum_36h',
 '樟树村雨量_sum_48h',
 '横洋雨量_std_12h',
 '民主村雨量_max_48h',
 '民主村雨量_min_24h',
 '民主村雨量_min_48h',
 '民主村雨量_sum_48h',
 '水东区间流域面雨量_max_24h',
 '水东区间流域面雨量_max_48h',
 '水东区间流域面雨量_std_12h',
 '水东区间流域面雨量_sum_3h',
 '水东区间流域面雨量_sum_6h',
 '水东区间流域面雨量_sum_36h',
 '水东区间流域面雨量_sum_48h',
 '水东坝上雨量_std_3h',
 '水东面雨量_max_36h',
 '水东面雨量_max_48h',
 '水东面雨量_std_48h',
 '水南村雨量_max_12h',
 '水南村雨量_max_48h',
 '水南村雨量_std_48h',
 '水南村雨量_sum_36h',
 '水南村雨量_sum_48h',
 '水口坝上(左岸)雨量_max_24h',
 '水口坝上(左岸)雨量_max_36h',
 '水口坝上(左岸)雨量_max_48h',
 '水口坝上(左岸)雨量_std_6h',
 '水口坝上(左岸)雨量_std_12h',
 '水口坝上(左岸)雨量_std_36h',
 '水口坝上(左岸)雨量_std_48h',
 '水口坝上雨量_max_36h',
 '水口坝上雨量_min_24h',
 '水口面雨量_max_24h',
 '水口面雨量_max_36h',
 '水口面雨量_max_48h',
 '水口面雨量_min_48h',
 '水口面雨量_sum_36h',
 '水口面雨量_sum_48h',
 '永坑村雨量_min_36h',
 '永坑村雨量_std_24h',
 '永坑村雨量_std_48h',
 '永坑村雨量_sum_48h',
 '沙坝下雨量_max_3h',
 '沙坝下雨量_max_48h',
 '沙坝下雨量_std_12h',
 '沙坝下雨量_std_48h',
 '沙溪流域面雨量_max_24h',
 '漈村雨量_sum_12h',
 '白叶山雨量_max_48h',
 '白叶山雨量_std_6h',
 '白叶山雨量_std_36h',
 '白叶山雨量_std_48h',
 '白叶山雨量_sum_48h',
 '白源雨量_max_36h',
 '白源雨量_max_48h',
 '白源雨量_std_3h',
 '白源雨量_std_6h',
 '白源雨量_std_48h',
 '石塔山雨量_max_24h',
 '石塔山雨量_max_48h',
 '石塔山雨量_std_48h',
 '石塔山雨量_sum_24h',
 '石塔山雨量_sum_48h',
 '福源小区雨量_std_3h',
 '福源小区雨量_sum_48h',
 '福记雨量_max_48h',
 '福记雨量_std_6h',
 '福记雨量_std_48h',
 '福记雨量_sum_48h',
 '端溪村雨量_max_36h',
 '端溪村雨量_max_48h',
 '端溪村雨量_std_6h',
 '端溪村雨量_sum_48h',
 '街面面雨量_max_48h',
 '街面面雨量_std_48h',
 '街面面雨量_sum_3h',
 '西城镇雨量_min_12h',
 '西城镇雨量_min_24h',
 '西城镇雨量_std_12h',
 '西津村雨量_max_48h',
 '西津村雨量_std_3h',
 '西津村雨量_std_6h',
 '西表村雨量_max_36h',
 '西表村雨量_max_48h',
 '西表村雨量_min_6h',
 '西表村雨量_std_3h',
 '西表村雨量_std_6h',
 '西表村雨量_std_12h',
 '西表村雨量_std_24h',
 '西表村雨量_std_36h',
 '西表村雨量_std_48h',
 '西表村雨量_sum_36h',
 '西表村雨量_sum_48h',
 '观华园雨量_sum_3h',
 '观华园雨量_sum_36h',
 '观华园雨量_sum_48h',
 '车洋坂雨量_std_12h',
 '车洋坂雨量_std_24h',
 '车洋坂雨量_std_48h',
 '车洋坂雨量_sum_36h',
 '车洋坂雨量_sum_48h',
 '里坪厂雨量_std_6h',
 '里洋村雨量_max_36h',
 '里洋村雨量_max_48h',
 '里洋村雨量_std_3h',
 '里洋村雨量_std_6h',
 '里洋村雨量_std_12h',
 '里洋村雨量_std_36h',
 '里洋村雨量_sum_12h',
 '里洋村雨量_sum_36h',
 '里洋村雨量_sum_48h',
 '金岭村雨量_max_36h',
 '金岭村雨量_max_48h',
 '金岭村雨量_std_3h',
 '金岭村雨量_std_6h',
 '金岭村雨量_std_48h',
 '金岭村雨量_sum_48h',
 '雍口实时面雨量_std_6h',
 '雍口实时面雨量_std_36h',
 '雍口面雨量_max_36h',
 '雍口面雨量_max_48h',
 '雍口面雨量_std_48h',
 '龙口村雨量_min_36h',
 '年份',
 '月份',
 '日',
 '小时',
 '季度',
 '水文时期编码',
 'v_sum',
 'v_mean',
 'v_std',
 'v_max',
 'v_min',
 'v_skew',
 'v_sum_diff',
 'v_mean_diff',
 'v_mean_roll_8',
 'v_mean_roll_16',
 'v_mean_roll_24',
 'v_mean_roll_32',
 'v_mean_roll_40',
 'v_mean_roll_48',
 'v_mean_roll_56',
 'v_mean_roll_64',
 'v_mean_roll_72',
 'v_mean_roll_80',
 'v_mean_roll_88',
 'v_mean_roll_96',
 'v_mean_roll_104',
 'v_mean_roll_112',
 'v_mean_roll_120',
 'v_mean_roll_128',
 'v_mean_roll_136',
 'v_mean_roll_144',
 'v_mean_roll_152',
 'v_mean_roll_160',
 'v_mean_roll_168',
 'v_mean_roll_176',
 'v_mean_roll_184',
 'v_mean_roll_192',
 'v_mean_roll_200',
 'v_mean_roll_208',
 'v_mean_roll_216',
 'v_mean_roll_224',
 'v_mean_roll_232',
 'v_mean_roll_240',
 'v_mean_roll_248',
 'v_mean_roll_256',
 'v_mean_roll_264',
 'v_mean_roll_272',
 'v_mean_roll_280',
 'v_mean_roll_288',
 'v_mean_roll_296',
 'v_mean_roll_304',
 'v_mean_roll_312',
 'v_mean_roll_320',
 'v_mean_roll_328',
 'v_mean_roll_336',
 'v_mean_roll_344',
 'v_mean_roll_352',
 'v_mean_roll_360',
 'v_mean_roll_368',
 'v_mean_roll_376',
 'v_mean_roll_384',
 'v_mean_roll_392',
 'v_mean_roll_400',
 'v_mean_roll_408',
 'v_mean_roll_416',
 'v_mean_roll_424',
 'v_mean_roll_432',
 'v_mean_roll_440',
 'v_mean_roll_448',
 'v_mean_roll_456',
 'v_mean_roll_464',
 'v_mean_roll_472',
 'v_mean_roll_480',
 'v_mean_roll_488',
 'v_mean_roll_496',
 'v_mean_roll_504',
 'v_mean_roll_512',
 'v_mean_roll_520',
 'v_mean_roll_528',
 'v_mean_roll_536',
 'v_mean_roll_544',
 'v_mean_roll_552',
 'v_mean_roll_560',
 'v_mean_roll_568',
 'v_mean_roll_576',
 'v_mean_roll_584',
 'v_mean_roll_592',
 'v_mean_roll_600',
 'v_mean_roll_608',
 'v_mean_roll_616',
 'v_mean_roll_624',
 'v_mean_roll_632',
 'v_mean_roll_640',
 'v_mean_roll_648',
 'v_mean_roll_656',
 'v_mean_roll_664',
 'v_mean_roll_672',
 'v_mean_roll_680',
 'v_mean_roll_688',
 'v_mean_roll_696',
 'v_mean_roll_704',
 'v_mean_roll_712',
 'v_mean_roll_720']

In [87]:
lgb_params = {
    'device': 'gpu',          # 启用GPU
    'boost_from_average': 'false',
    'boost': 'gbdt',
    'metric': 'rmse',
    'max_depth': 7,
    'num_leaves': 2**7-1,
    'objective': 'regression',
    'min_child_weight': 16,
    'min_data_in_leaf': 6,
    'min_split_gain': 0.7,
    'bagging_fraction': 0.82,
    'feature_fraction': 0.74,
    'reg_lambda': 2.65,
    'reg_alpha': 3.85,
    'bagging_freq': 1,
    'seed': 2023,
    'nthread': 8,
    'n_jobs': 8,
    'verbose': -1
}


In [88]:
from sklearn.metrics import mean_squared_error
import numpy as np
## 获取lightgbm的特征重要度，这里应用的lightgbm的随机森林算法和论文内的算法逻辑保持一致
def get_feature_importances(data,y, shuffle=False, train_features = [],seed=None,params={}):
    # 是否要打乱
    if shuffle:
        # Here you could as well use a binomial distribution
        y = y.copy().sample(frac=1.0)
    
    # Fit LightGBM in RF mode, yes it's quicker than sklearn RandomForest
    dtrain = lgb.Dataset(data[train_features], y, free_raw_data=False )
    
    # Fit the model
    clf = lgb.train(params=params, train_set=dtrain, num_boost_round=200)

    # Get feature importances
    imp_df = pd.DataFrame()
    imp_df["feature"] = list(train_features)
    imp_df["importance_gain"] = clf.feature_importance(importance_type='gain')
    imp_df["importance_split"] = clf.feature_importance(importance_type='split')

    # 计算RMSE训练分数（添加sqrt确保正确性）
    predictions = clf.predict(data[train_features])
    imp_df['trn_score'] = np.sqrt(mean_squared_error(y, predictions))  # RMSE
    
    return imp_df

In [89]:
# use_df = 设置模型训练的dataFrame
use_df = df_train.drop(columns='时间')

In [90]:
# 标签数据
label = 'FLAG'
# 真实响应变量下的特征重要度
feats_use = feature_selected
actual_imp_df = get_feature_importances(data=use_df, y=use_df[label],shuffle=False,train_features=feats_use,params=lgb_params)

[LightGBM] [Warning] num_threads is set with nthread=8, will be overridden by n_jobs=8. Current value: num_threads=8


In [91]:
import time
start = time.time()
# null importance 变量
null_imp_df = pd.DataFrame()
# 运行次数50次
nb_runs = 50
dsp = ''
# 运算Null importance
for i in range(nb_runs):
    print(i)
    # Get current run importances
    imp_df = get_feature_importances(data=use_df, y=use_df[label], shuffle=True,train_features=feats_use,params=lgb_params)
    imp_df['run'] = i + 1 
    # Concat the latest importances with the old ones
    null_imp_df = pd.concat([null_imp_df, imp_df], axis=0)
    # Erase previous message
#     for l in range(len(dsp)):
#         print('\b', end='', flush=True)
    # Display current run and time used
    spent = (time.time() - start) / 60
    dsp = 'Done with %4d of %4d (Spent %5.1f min)' % (i + 1, nb_runs, spent)
    print(dsp, end='', flush=True)

0
[LightGBM] [Warning] num_threads is set with nthread=8, will be overridden by n_jobs=8. Current value: num_threads=8
Done with    1 of   50 (Spent   0.1 min)1
[LightGBM] [Warning] num_threads is set with nthread=8, will be overridden by n_jobs=8. Current value: num_threads=8
Done with    2 of   50 (Spent   0.1 min)2
[LightGBM] [Warning] num_threads is set with nthread=8, will be overridden by n_jobs=8. Current value: num_threads=8
Done with    3 of   50 (Spent   0.2 min)3
[LightGBM] [Warning] num_threads is set with nthread=8, will be overridden by n_jobs=8. Current value: num_threads=8
Done with    4 of   50 (Spent   0.2 min)4
[LightGBM] [Warning] num_threads is set with nthread=8, will be overridden by n_jobs=8. Current value: num_threads=8
Done with    5 of   50 (Spent   0.3 min)5
[LightGBM] [Warning] num_threads is set with nthread=8, will be overridden by n_jobs=8. Current value: num_threads=8
Done with    6 of   50 (Spent   0.3 min)6
[LightGBM] [Warning] num_threads is set with

In [92]:
feature_scores = []
for _f in actual_imp_df['feature'].unique():

    f_null_imps_gain = null_imp_df.loc[null_imp_df['feature'] == _f, 'importance_gain'].values
    f_act_imps_gain = actual_imp_df.loc[actual_imp_df['feature'] == _f, 'importance_gain'].mean()
    gain_score = np.log(1e-10 + f_act_imps_gain / (1 + np.percentile(f_null_imps_gain, 75)))  # Avoid didvide by zero
    f_null_imps_split = null_imp_df.loc[null_imp_df['feature'] == _f, 'importance_split'].values
    f_act_imps_split = actual_imp_df.loc[actual_imp_df['feature'] == _f, 'importance_split'].mean()
    split_score = np.log(1e-10 + f_act_imps_split / (1 + np.percentile(f_null_imps_split, 75)))  # Avoid didvide by zero
    feature_scores.append((_f, split_score, gain_score))

scores_df = pd.DataFrame(feature_scores, columns=['feature', 'split_score', 'gain_score'])

In [93]:
feats = scores_df['feature'][(scores_df['split_score']>0)&(scores_df['gain_score']>0)].tolist()

In [94]:
feats

['下保村雨量_max_48h',
 '下保村雨量_std_12h',
 '下洋坂雨量_std_6h',
 '均中村雨量_max_36h',
 '均中村雨量_std_48h',
 '均中村雨量_sum_48h',
 '外洋村雨量_max_12h',
 '外洋村雨量_std_12h',
 '外洋村雨量_std_48h',
 '外洋村雨量_sum_48h',
 '大横镇曲楼甲新农村雨量_max_48h',
 '大横镇曲楼甲新农村雨量_std_48h',
 '大横镇曲楼甲新农村雨量_sum_48h',
 '富屯溪流域面雨量_min_12h',
 '富屯溪流域面雨量_sum_48h',
 '山下乡雨量_max_48h',
 '山下乡雨量_sum_48h',
 '岭坪雨量_max_48h',
 '岭坪雨量_std_36h',
 '建溪流域面雨量_max_48h',
 '建溪流域面雨量_sum_48h',
 '文峰村雨量_std_6h',
 '文峰村雨量_std_36h',
 '新源村雨量_max_48h',
 '杉坜雨量_std_6h',
 '杉坜雨量_std_48h',
 '杉坜雨量_sum_48h',
 '林后岗雨量_min_24h',
 '林后岗雨量_min_48h',
 '林后岗雨量_std_3h',
 '林后岗雨量_std_12h',
 '林后岗雨量_sum_48h',
 '柯坑村雨量_max_48h',
 '桥南村雨量_max_6h',
 '桥南村雨量_max_12h',
 '桥南村雨量_min_24h',
 '桥南村雨量_std_6h',
 '桥南村雨量_std_12h',
 '桥南村雨量_std_48h',
 '桥南村雨量_sum_48h',
 '梅园小区雨量_std_24h',
 '梅园小区雨量_sum_24h',
 '樟树村雨量_max_48h',
 '樟树村雨量_std_12h',
 '樟树村雨量_sum_36h',
 '樟树村雨量_sum_48h',
 '水东区间流域面雨量_max_48h',
 '水东区间流域面雨量_std_12h',
 '水南村雨量_sum_36h',
 '水南村雨量_sum_48h',
 '水口坝上(左岸)雨量_max_36h',
 '水口坝上(左岸)雨量_max_48h',
 '水口坝上(左岸)雨量_std_6h',
 '水口坝

In [95]:
import pickle
pickle.dump(feats, open('./20250328凌晨跑的np特征' + '.pkl', 'wb'))